<a href="https://colab.research.google.com/github/johntanas/it1244project/blob/main/code/model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
path_to_evm_label="/content/expo_moving_labels.csv"
path_to_app="/content/cleaned_application.csv"
cleaned_app_df=pd.read_csv(path_to_app,index_col=0)
ewm_df_status=pd.read_csv(path_to_evm_label,index_col=0)

In [4]:
dep_var="label"
random_state=42

In [75]:
labels_to_int_evm={"bad":1,"neutral":2,"good":3}

In [77]:
int_to_labels_evm={v:k for k,v in enumerate(labels_to_int_evm)}

In [61]:
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from sklearn.model_selection import ShuffleSplit
from fastai.tabular.all import *
from sklearn.model_selection import KFold

In [78]:
df_nn=ewm_df_status.merge(cleaned_app_df,how="inner",on="id").replace({"label":labels_to_int_evm})

In [79]:
df_nn_final=df_nn.drop(["status","id"],axis=1)

In [80]:
cont_nn,cat_nn = cont_cat_split(df_nn_final, max_card=1000, dep_var=dep_var)

In [81]:
sss = ShuffleSplit(n_splits=1, test_size=0.2)
train_idx, valid_idx = next(sss.split(train_df))
splits = (list(train_idx),list(valid_idx))

In [96]:
procs_nn = [Categorify, Normalize]
to_nn = TabularPandas(df_nn_final, procs_nn, cat_nn, cont_nn,
                      splits=splits, y_names=dep_var,
                    y_block=MultiCategoryBlock())

In [97]:
dls = to_nn.dataloaders(1024)

In [98]:
df_nn_final[cat_nn].nunique()

gender              2
own_car             2
own_realty          2
num_child           9
income_type         5
education_level     5
family_status       5
house_type          6
work_phone          2
phone               2
email               2
job                19
dtype: int64

In [99]:
y = to_nn.train.y

In [100]:
learn = tabular_learner(dls, layers=[500,250],loss_func=FocalLossFlat(reduction='none',axis=1),metrics=[accuracy,F1Score(),HammingLoss()])

In [101]:
learn.lr_find()

RuntimeError: ignored

In [ ]:
learn.fit_one_cycle(5, 2e-2)  

It seems that our model does better than a naive guess at statistically insiginificant level

It might be due to poor train test spliting. Lets try Kfolds

In [ ]:
kf=KFold(n_splits=5, shuffle=True, random_state=random_state)

In [ ]:
fold=1
for train_index, test_index in kf.split(df_nn_final):
  s=(list(train_index), list(test_index))
  to_nn = TabularPandas(df_nn_final, procs_nn, cat_nn, cont_nn,
                      splits=s, y_names=dep_var)
  dls = to_nn.dataloaders(1024)
  y = to_nn.train.y
  learn = tabular_learner(dls, layers=[500,250],loss_func=BCEWithLogitsLossFlat())
  learn.fit_one_cycle(5, 1e-2)  
  preds,targs = learn.get_preds()
  print(f"Accuracy of fold {fold}\n")
  print(hamming_distance_accuracy(preds, np.array(targs)).item())
  fold+=1

In [ ]:
class_0 = df_nn_final[df_nn_final['label'] == 0]
class_1 = df_nn_final[df_nn_final['label'] == 1]

In [ ]:
class_count_0=len(class_0)

In [ ]:
class_1_under = class_1.sample(class_count_0)

test_under = pd.concat([class_1_under, class_0], axis=0)

print("total class of 1 and 0:",test_under['label'].value_counts())# plot the count after under-sampeling
test_under['label'].value_counts().plot(kind='bar', title='count (target)')

In [ ]:
sss = ShuffleSplit(n_splits=1, test_size=0.2)
train_idx, valid_idx = next(sss.split(test_under))
splits = (list(train_idx),list(valid_idx))

In [ ]:
to_nn = TabularPandas(test_under, procs_nn, cat_nn, cont_nn,
                      splits=splits, y_names=dep_var)
dls = to_nn.dataloaders(1024)
learn = tabular_learner(dls, layers=[500,250],loss_func=BCEWithLogitsLossFlat(),metrics=[accuracy,F1Score(),HammingLoss()])
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, 0.0005)  

the model is learning nothing! no data correlation with labels!